In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import UndefinedMetricWarning

In [2]:
# Loading the datasets provided by the user
data = pd.read_csv('Base.csv')

In [3]:
# Data Preprocessing
# Separate features and target
X = data.drop(columns=['fraud_bool'])
y = data['fraud_bool'].values.ravel()  # Ensure target is a 1D array

# Convert categorical variables using one-hot encoding
X_encoded = pd.get_dummies(X, drop_first=True)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

In [ ]:
# Logistic Regression with LASSO (L1 penalty) and 10-fold cross-validation
logit_lasso = LogisticRegressionCV(
    penalty='l1', 
    solver='liblinear',  # Supports L1 penalty
    cv=10, 
    scoring='accuracy',
    max_iter=100
)

# Fit model
logit_lasso.fit(X_train, y_train)

# Make predictions
y_pred = logit_lasso.predict(X_test)
print(y_pred)

In [ ]:
# Performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {fscore:.4f}')

In [ ]:
# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion Matrix Visualization
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Coefficients summary
coefficients_df = pd.DataFrame(model.coef_, columns=X_encoded.columns)
print("\nModel Coefficients:")
print(coefficients_df.T)